**1**. Connect to the SQLite3 database at `data/faculty.db` using the `sqlite` package or `ipython-sql` magic functions.

In [31]:
%load_ext sql
%sql sqlite:///faculty.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @faculty.db'

In [32]:
%%sql
SELECT * 
FROM sqlite_master

 * sqlite:///faculty.db
Done.


type,name,tbl_name,rootpage,sql
table,person,person,2,"CREATE TABLE person ( ""index"" BIGINT, person_id BIGINT, first TEXT, last TEXT, age BIGINT, height FLOAT, weight BIGINT, country_id TEXT, gender_id BIGINT)"
index,ix_person_index,person,3,"CREATE INDEX ix_person_index ON person (""index"")"
table,confidential,confidential,18,"CREATE TABLE confidential ( ""index"" BIGINT, person_id BIGINT, salary BIGINT)"
index,ix_confidential_index,confidential,24,"CREATE INDEX ix_confidential_index ON confidential (""index"")"
table,person_language,person_language,33,"CREATE TABLE person_language ( ""index"" BIGINT, person_id BIGINT, language_id BIGINT)"
index,ix_person_language_index,person_language,34,"CREATE INDEX ix_person_language_index ON person_language (""index"")"
table,language,language,50,"CREATE TABLE language ( ""index"" BIGINT, language_id BIGINT, language_name TEXT)"
index,ix_language_index,language,54,"CREATE INDEX ix_language_index ON language (""index"")"
table,gender,gender,55,"CREATE TABLE gender ( ""index"" BIGINT, gender_id BIGINT, gender TEXT)"
index,ix_gender_index,gender,56,"CREATE INDEX ix_gender_index ON gender (""index"")"


2. Find the youngest and oldest faculty member of each gender.

Youngest and then oldest males.

In [33]:
%%sql
SELECT first, last, age
FROM person
WHERE gender_id = 0
ORDER BY age
LIMIT 1

 * sqlite:///faculty.db
Done.


first,last,age
Antoine,Beard,16


In [34]:
%%sql
SELECT first, last, age
FROM person
WHERE gender_id = 0
ORDER BY age DESC
LIMIT 1


 * sqlite:///faculty.db
Done.


first,last,age
Antony,Gregory,66


Youngest and then oldest females. 

In [35]:
%%sql
SELECT first, last, age
FROM person
WHERE gender_id = 1
ORDER BY age
LIMIT 1


 * sqlite:///faculty.db
Done.


first,last,age
Chery,Becker,16


In [36]:
%%sql
SELECT first, last, age
FROM person
WHERE gender_id = 1
ORDER BY age DESC
LIMIT 1

 * sqlite:///faculty.db
Done.


first,last,age
Alyse,Gamble,66


3. Find the median age of the faculty members who know Python.

As SQLite3 does not provide a median function, you can create a User Defined Function (UDF) to do this. See [documentation](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection.create_function).

In [96]:
%%sql
SELECT avg(age)
FROM (SELECT age
      FROM person
      ORDER BY age
      WHERE 
      LIMIT 2 - (SELECT COUNT(*) FROM person) % 2    -- odd 1, even 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM person))

 * sqlite:///faculty.db
Done.


avg(age)
41.0


4. Arrange countries by the average age of faculty in descending order. Countries are only included in the table if there are at least 3 faculty members from that country.

In [42]:
%%sql
SELECT country_id, avg(age)
FROM person
GROUP BY country_id
ORDER BY avg(age) desc

 * sqlite:///faculty.db
Done.


country_id,avg(age)
HK,48.785714285714285
UY,45.36842105263158
ME,44.77777777777778
IS,44.26315789473684
UA,44.17391304347826
GB,44.1025641025641
CH,43.357142857142854
EE,43.17948717948718
BE,43.08108108108108
CL,42.83783783783784


5. Which country has the most highest average body mass index (BMII) among the faculty? Recall that BMI is weight (kg) / (height (m))^2.

In [46]:
%%sql
SELECT country_id, avg(weight/(height*height))
FROM person
GROUP BY country_id
ORDER BY avg(weight/(height*height)) desc
LIMIT 1


 * sqlite:///faculty.db
Done.


country_id,avg(weight/(height*height))
HK,24.54940412907561


6. Do obese faculty (BMI > 30) know more languages on average than non-obese faculty?

Obese Faculty know fewer languages on average!

In [87]:
%%sql
SELECT AVG (langs)
FROM
    (SELECT COUNT(DISTINCT(person_language.language_id)) as langs
    FROM person_language
    JOIN person
    ON person.person_id = person_language.person_id
    WHERE(person.weight/(person.height*person.height))>30
    GROUP BY person_language.person_id
    ORDER BY COUNT(DISTINCT(person_language.language_id)) desc)

 * sqlite:///faculty.db
Done.


AVG (langs)
1.9508196721311475


Non-obese faculty know lightly more languages.

In [90]:
%%sql
SELECT AVG (langs)
FROM
    (SELECT COUNT(DISTINCT(person_language.language_id)) as langs
    FROM person_language
    JOIN person
    ON person.person_id = person_language.person_id
    WHERE(person.weight/(person.height*person.height))<30
    GROUP BY person_language.person_id
    ORDER BY COUNT(DISTINCT(person_language.language_id)) desc)

 * sqlite:///faculty.db
Done.


AVG (langs)
1.9644230769230768
